In [ ]:
from __future__ import unicode_literals
from polyglot.text import Text
import collections
import pandas as pd
import numpy as np
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from reldi_tokeniser import tokeniser
from reldi_tagger import tagger

import os


# -*- coding: utf-8 -*-

from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 95% !important; }</style>"))

In [ ]:
data = pd.read_csv('SerbMR-2C.csv', encoding='utf8')

In [ ]:
def removePunctuation(asd):
    reg = re.compile(r'[^\w\s|.]')
    new = []
    for x in asd:
        if not reg.search(x):
            new.append(x.lower())
    return new

tokens_out = []
for row in data.iterrows():
    text, polarity = row[1]['Text'], row[1]['class-att']
    asd = tokeniser.tok(text, 'sr')
    asd = removePunctuation(asd.split('\n')[:-1])

    if polarity == 'POSITIVE':
        polarity = 1
    elif polarity == 'NEGATIVE':
        polarity = -1
    else: polarity = 0
    tokens_out.append((asd, polarity))

pickle.dump(tokens_out, open('tokenised_SerbMR-2C_with_periods.pickle', 'wb'), 2)

In [ ]:
mp = {}
for i, row in data.iterrows():
    text = Text(row['Text'], hint_language_code='sr')
    pol = []
    for w in text.words:
        pol.append(w.polarity)
        mp[w] = w.polarity
    print(i, np.mean(pol), row['class-att'])

In [ ]:
from tqdm import tqdm
with open('tokenised_SerbMR-3C.pickle') as f:
    d = pickle.load(f)

lem_output = []
for entry in tqdm(d):
    asd = tagger.lemantise(entry[0], 'sr')
    lem_output.append((asd, entry[1]))

pickle.dump(lem_output, open('lemmantised_SerbMR-3C.p', 'wb'), 2)

In [ ]:
with open('hr.sent.pkl') as f:
    d = pickle.load(f)

with open('sh.lexicon', 'a') as f:
    tmp = []
    for ent in zip(d[0], d[1]):
        f.write(u"{} {}".format(ent[0], ent[1][0]).encode('utf-8').strip() + os.linesep)
    
# pickle.dump(tmp, open('sh.lexicon', 'w'), 0)

In [ ]:
words = {}
with open('sr.lexicon') as f:
    for line in f.readlines():
        l = line.decode('utf-8').split(' ')
        words[tagger.lemantise([l[0].strip()], 'sr')[0]] = l[1].strip()
        
with open('sr.final.lexicon', 'w') as f:
    for k, v in words.items():
        f.write(u"{} {}".format(k, v).encode('utf-8').strip() + os.linesep)